In [1]:
import sys, pkgutil

print("sys.executable:", sys.executable)


sys.executable: /Users/rahuljangra/Downloads/CodeWithRahul01/deepstudysession/bin/python


In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
# /Users/rahuljangra/Downloads/CodeWithRahul01/studysession/bin/python -m pip install tensorflow

TensorFlow version: 2.16.2


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder
import pickle

In [4]:
df = pd.read_csv("/Users/rahuljangra/Downloads/CodeWithRahul01/Notes /annclassification/Churn_Modelling.csv")

In [5]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
#Preprocessing 
## Dropping Unnecessary Columns
df = df.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

In [7]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
# Encoding the Categorical Columns 
label_Encoder = LabelEncoder()
df["Gender"] = label_Encoder.fit_transform(df["Gender"])
# df["Geography"] = Encoder.fit_transform(df["Geography"])

In [9]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [10]:
pd.unique(df["Geography"])
# df.head()

array(['France', 'Spain', 'Germany'], dtype=object)

In [11]:
# encoded_df = pd.get_dummies(df, columns = ["Geography"],prefix= "is_")

In [12]:
# encoded_df.head()

In [13]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()
geo_encoded = one_hot_encoder.fit_transform(df[["Geography"]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns= one_hot_encoder.get_feature_names_out(["Geography"]))
geo_encoded_df.head()
final_df = pd.concat([df.drop("Geography", axis=1), geo_encoded_df], axis=1)

In [14]:
final_df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [15]:
# save encoder to pickle file
with open("label_encoder.pkl", "wb") as file:
  pickle.dump(label_Encoder, file)
with open("One_hot_encoder.pkl", "wb") as file:
  pickle.dump(one_hot_encoder, file)

In [16]:
X = final_df.drop("Exited", axis=1).values
y = final_df["Exited"].values

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 101)

In [18]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
with open("scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

## ANN Implementation 

In [20]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [21]:
model = Sequential([
  Dense(64 , activation = "relu",input_shape = (X_train.shape[1],)),
  Dense(32 , activation = "relu"),
  Dense(1, activation = "sigmoid")
])

/Users/rahuljangra/Downloads/CodeWithRahul01/deepstudysession/lib/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-11-27 20:55:13.927310: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-11-27 20:55:13.927502: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-11-27 20:55:13.927509: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-11-27 20:55:13.927545: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-27 20:55:13.927558: I tensorflow/core/common_runtime/pluggable_device/plugga

In [22]:
opt = tf.keras.optimizers.Adam(learning_rate = 0.01)

In [23]:
# compile the model 
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics=["accuracy"])

In [24]:
#Set up TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [25]:
# set up early stopping 
early_stopping_callback = EarlyStopping(monitor = "val_loss", patience= 10, restore_best_weights=True)

In [26]:
history = model.fit(
  X_train, y_train, validation_data = (X_test,y_test),
  epochs = 100, callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100


2025-11-27 20:55:14.617414: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7850 - loss: 0.4731 - val_accuracy: 0.8035 - val_loss: 0.4295
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8096 - loss: 0.4348 - val_accuracy: 0.8150 - val_loss: 0.4382
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8111 - loss: 0.4384 - val_accuracy: 0.7995 - val_loss: 0.4309
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8092 - loss: 0.4475 - val_accuracy: 0.8160 - val_loss: 0.5745
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8043 - loss: 0.4768 - val_accuracy: 0.8100 - val_loss: 0.4453
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8021 - loss: 0.4635 - val_accuracy: 0.8010 - val_loss: 0.4448
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8024 - loss: 0.4661 - val_accuracy: 0.8055 - val_loss: 0.4361
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8049 - loss: 0.4669 - val_accuracy

In [27]:
model.save("model.h5")

In [28]:
%load_ext tensorboard 

In [29]:
%tensorboard --logdir logs/fit